[View in Colaboratory](https://colab.research.google.com/github/reachnivi/Machine-Learning-Projects/blob/master/Disaster_prediction_Hackerearth.ipynb)

In [0]:
import pandas as pd
import numpy as np


In [2]:
from google.colab import files
def getLocalFiles():
    _files = files.upload()
    if len(_files) >0:
       for k,v in _files.items():
         open(k,'wb').write(v)
getLocalFiles()

In [0]:
from google.colab import files

In [6]:
!wget --header 'Host: he-s3.s3.amazonaws.com' --user-agent 'Mozilla/5.0 (Windows NT 6.1; rv:61.0) Gecko/20100101 Firefox/61.0' --header 'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8' --header 'Accept-Language: en-US,en;q=0.5' --header 'Upgrade-Insecure-Requests: 1' 'https://he-s3.s3.amazonaws.com/media/hackathon/machine-learning-challenge-6-1/predict-the-energy-used-612632a9-3f496e7f/a490e594-6-Dataset.zip' --output-document 'a490e594-6-Dataset.zip'

--2018-08-02 15:50:26--  https://he-s3.s3.amazonaws.com/media/hackathon/machine-learning-challenge-6-1/predict-the-energy-used-612632a9-3f496e7f/a490e594-6-Dataset.zip
Resolving he-s3.s3.amazonaws.com (he-s3.s3.amazonaws.com)... 52.219.40.132
Connecting to he-s3.s3.amazonaws.com (he-s3.s3.amazonaws.com)|52.219.40.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 33194548 (32M) [application/zip]
Saving to: ‘a490e594-6-Dataset.zip’

a490e594-6-Dataset. 100%[===================>]  31.66M  2.75MB/s    in 40s     

2018-08-02 15:51:06 (814 KB/s) - ‘a490e594-6-Dataset.zip’ saved [33194548/33194548]



In [7]:
!ls

a490e594-6-Dataset.zip	datalab


In [9]:
!unzip a490e594-6-Dataset.zip


Archive:  a490e594-6-Dataset.zip
  inflating: Dataset/Building_Ownership_Use.csv  
  inflating: Dataset/Building_Structure.csv  
  inflating: Dataset/ReadMe.md       
  inflating: Dataset/sample_submission.csv  
  inflating: Dataset/test.csv        
  inflating: Dataset/train.csv       


In [0]:
train = pd.read_csv("Dataset/train.csv")
test = pd.read_csv("Dataset/test.csv")

In [12]:
train.shape

(631761, 14)

In [16]:
train.columns

Index(['area_assesed', 'building_id', 'damage_grade', 'district_id',
       'has_geotechnical_risk', 'has_geotechnical_risk_fault_crack',
       'has_geotechnical_risk_flood', 'has_geotechnical_risk_land_settlement',
       'has_geotechnical_risk_landslide', 'has_geotechnical_risk_liquefaction',
       'has_geotechnical_risk_other', 'has_geotechnical_risk_rock_fall',
       'has_repair_started', 'vdcmun_id'],
      dtype='object')

In [13]:
test.shape

(421175, 13)

In [15]:
BS = pd.read_csv("Dataset/Building_Structure.csv")
BO = pd.read_csv("Dataset/Building_Ownership_Use.csv")
print(BS.shape)
print(BO.shape)

(1052948, 29)
(1052948, 17)


In [0]:
merged = pd.merge(BS,BO,on="building_id")

In [0]:
train_merged = pd.merge(train,merged,on="building_id",how="left")

In [0]:
test_merged = pd.merge(test,merged,on="building_id",how="left")

In [20]:
train_merged.dtypes.value_counts()

int64      42
object     12
float64     4
dtype: int64

In [22]:
train_merged.select_dtypes(include=["object"]).columns

Index(['area_assesed', 'building_id', 'damage_grade', 'land_surface_condition',
       'foundation_type', 'roof_type', 'ground_floor_type', 'other_floor_type',
       'position', 'plan_configuration', 'condition_post_eq',
       'legal_ownership_status'],
      dtype='object')

In [0]:
df.head()


,area_assesed,damage_grade,district_id,has_geotechnical_risk,has_geotechnical_risk_fault_crack,has_geotechnical_risk_flood,has_geotechnical_risk_land_settlement,has_geotechnical_risk_landslide,has_geotechnical_risk_liquefaction,has_geotechnical_risk_other,has_geotechnical_risk_rock_fall,has_repair_started,vdcmun_id
building_id,,,,,,,,,,,,,
24385bfd2a2,Both,Grade 4,24,0.0,0,0,0,0,0,0,0,0.0,2489
405d1bbebbf,Both,Grade 2,44,0.0,0,0,0,0,0,0,0,1.0,4423
351d9bc71f6,Both,Grade 1,36,0.0,0,0,0,0,0,0,0,0.0,3650
2be3a971166,Building removed,Grade 5,30,0.0,0,0,0,0,0,0,0,0.0,3016
34c7d073ea6,Both,Grade 3,36,0.0,0,0,0,0,0,0,0,0.0,3627


Reading other datasets

In [0]:
b_str = pd.read_csv("Building_Structure.csv")
b_own = pd.read_csv("Building_Ownership_Use.csv")

join the data

In [0]:
df_test = pd.read_csv("test.csv")


In [0]:
res = pd.merge(b_str,b_own,on="building_id")

MemoryError: 